In [20]:
from bids.modeling import transformations as transform
from bids.variables import SparseRunVariable, DenseRunVariable
from bids.variables.entities import RunInfo
from bids.variables.collections import BIDSRunVariableCollection
from bids.layout import BIDSLayout
import math
from os.path import join, sep
from bids.tests import get_test_data_path
import numpy as np
import pandas as pd

In [22]:
SUBJECTS = ['01', '02']
NRUNS = 3
SCAN_LENGTH = 480

cached_collections = {}

if 'ds005' not in cached_collections:
    layout_path = join(get_test_data_path(), 'ds005')
    layout = BIDSLayout(layout_path)
    cached_collections['ds005'] = layout.get_collections(
        'run',
        types=['events'],
        scan_length=SCAN_LENGTH,
        merge=True,
        sampling_rate=10,
        subject=SUBJECTS
    )
    
collection = cached_collections['ds005'].clone()

/home/remi/github/pybids/bids/layout/models.py:159: SAWarning: Dialect sqlite:pysqlite will not make use of SQL compilation caching as it does not set the 'supports_statement_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Dialect maintainers should seek to set this attribute to True after appropriate development and testing for SQLAlchemy 1.4 caching support.   Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  result = session.query(Config).filter_by(name=config['name']).first()
/home/remi/github/pybids/bids/variables/entities.py:245: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.index = self.index.append(node_row, ignore_index=True)
/home/remi/github/pybids/bids/variables/entities

In [23]:
collection.variables

{'parametric gain': <bids.variables.variables.SparseRunVariable at 0x7f2e86133a00>,
 'trial_type': <bids.variables.variables.SparseRunVariable at 0x7f2e83e668b0>,
 'loss': <bids.variables.variables.SparseRunVariable at 0x7f2e853f59d0>,
 'PTval': <bids.variables.variables.SparseRunVariable at 0x7f2e83e0da00>,
 'respcat': <bids.variables.variables.SparseRunVariable at 0x7f2e83e0d7f0>,
 'gain': <bids.variables.variables.SparseRunVariable at 0x7f2e83e53310>,
 'RT': <bids.variables.variables.SparseRunVariable at 0x7f2e83e3ca60>,
 'respnum': <bids.variables.variables.SparseRunVariable at 0x7f2e83e3c6a0>}

In [24]:
dense_rt = collection.variables['RT'].to_dense(collection.sampling_rate)
assert len(dense_rt.values) == math.ceil(len(SUBJECTS) * NRUNS * SCAN_LENGTH * collection.sampling_rate)
transform.Rename(collections, 'RT', output='reaction_time')
assert 'reaction_time' in collection.variables
assert 'RT' not in collection.variables
col = collection.variables['reaction_time']
assert col.name == 'reaction_time'
assert col.onset.max() == 476

ValueError: No variable named 'RT' found in this collection. Available names are ['parametric gain', 'trial_type', 'loss', 'PTval', 'respcat', 'gain', 'respnum', 'reaction_time'].

In [25]:
data = pd.DataFrame({
    'onset': [10, 20],
    'duration': [0, 0],
    'amplitude': [1, 1]
})
run_info = [RunInfo({'subject': '01'}, 20, 2, 'dummy.nii.gz', 10)]
var = SparseRunVariable(
    name='var', data=data, run_info=run_info, source='events')
coll = BIDSRunVariableCollection([var])
transform.ToDense(coll, 'var', output='var_dense')
transform.Convolve(coll, 'var', output='var_hrf')
transform.Convolve(coll, 'var_dense', output='var_dense_hrf')

/home/remi/github/pybids/bids/variables/variables.py:409: UserWarning: The onset time of a variable seems to exceed the runs duration, hence runs are incremented by one internally.
  warnings.warn("The onset time of a variable seems to exceed "


In [26]:
c = collection
transform.Sum(
    collection, variables=['parametric gain', 'gain'], output='sum')
res = c['sum'].values
target = c['parametric gain'].values + c['gain'].values
assert np.array_equal(res, target)
transform.Sum(
    collection,
    variables=['parametric gain', 'gain'], output='sum', weights=[2, 2])
assert np.array_equal(c['sum'].values, target * 2)